In [1]:
# importing required libraries
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data/processed_data_10.csv')
df.head(3)

,id,job_geo_coordinate,Latitude,Longitude,installers_required,expected_job_time,est_installation_date,pref_date,pref_time_window,pref_day,pref_installer,installation_date,arrival_start_time,arrival_end_time,INSTALLER_IDs
0,2223,"-34.9007279,138.5199928",-34.900728,138.519993,1,150,15/01/2023,15/02/2023,"10:00,12:30",1.0,2.0,15/01/2023,08:25:00,10:55:00,1.0
1,2363,"-34.9336734,138.5141918",-34.933673,138.514192,1,120,17/01/2023,NaN,NaN,NaN,2.0,15/01/2023,11:02:00,01:02:00,1.0
2,2370,"-34.8408831,138.6441389",-34.840883,138.644139,1,8,10/01/2023,NaN,NaN,1.0,NaN,15/01/2023,08:00:00,10:00:00,NaN


In [5]:
df_raw = df.drop_duplicates(keep='first',ignore_index=True)
# Keeping only the important variables for now
imp_variables = ['id','job_geo_coordinate','installers_required','expected_job_time','est_installation_date','pref_date','pref_day','pref_installer','pref_time_window','installation_date','arrival_start_time','arrival_end_time','INSTALLER_IDs']
temp_df = df_raw[imp_variables]
df_clean = temp_df.drop_duplicates(subset='id', keep="first",ignore_index=True)

In [6]:
# Calculate travel time and Increase num of installers where exp job time >= 480
for i in range(len(df_clean)):
    if (df_clean.loc[i,'expected_job_time'] >= 480):
        df_clean.loc[i,'expected_job_time'] = np.ceil(df_clean.loc[i,'expected_job_time']/2)
        df_clean.loc[i,'installers_required']*=2
df_clean = df_clean[df_clean["job_geo_coordinate"].notnull()]
df_clean.reset_index(inplace = True,drop=True)

In [7]:
## !!IMPORTANT CONSTRAINT
# df_clean['pref_day'] = df_clean['pref_day']-1 ## Currently assuming original data has day_of_the_week from 1 to 7 --> So remove this line if not the case

In [8]:
df_clean[['Latitude','Longitude']] = df_clean["job_geo_coordinate"].str.split(',',expand=True) 
cols=list(df_clean.columns)
cols=cols[:2]+cols[-2:]+cols[2:-2]
df_clean=df_clean[cols]
df_clean.rename(columns={'INSTALLER_IDs': 'installer_ids'}, inplace=True)
df_clean = df_clean.sort_values(by=['installers_required','id'], ascending=[True,True],ignore_index=True) # Sorting according to installers_required first and then ids
df_clean['status'] = None # Adding a status column to keep track of planned jobs

In [9]:
df_clean.to_csv('data/processed_data_10.csv',index=False)